In [12]:
from glob import glob
import cv2,os
import SimpleITK as sitk
import numpy as np
from tqdm import tqdm
from operator import itemgetter
import pandas as pd 
from shutil import copyfile

import random,json


path1 = 'CTDircadb1/test/pred'
path2 = 'CTDircadb1/test/mask'
path3 = 'CTDircadb1/test/origin'
path4 = 'hospital_data_clean/png/pred'
path5 = 'hospital_data_clean/png/origin'
# path6 = 'LIDC_nodul/origin'

In [5]:
# 取一个hospital 数据集volumn的函数

def get_nii(volumn,path_in='hospital_data_clean/png/pred'):
   '''
   volumn:取出哪一个volumn
   path_in:数据集地址
   '''
   path_in = path_in.rstrip('/')
   a = os.listdir(path_in)
   a.sort()
   list_res = []
   for i in range(volumn,volumn+1):
      list_temp = [j for j in a if 'id{:0>2d}'.format(i) in j]
      list_temp.sort(key=lambda x:int(x.split('_')[1].split('.')[0]))
      list_res+=list_temp
      
   a = list_res
   c = path_in + '/' + pd.Series(a) 
   c.tolist()
   
   from tools import fold2nii

   fold2nii(c,path_in+volumn)

In [4]:
# 取volumn=12来看看情况

v = 12

for name in ['mask_bone','origin']:
    
    path_temp = 'hospital_data_clean/png/{}'.format(name)
    get_nii(volumn = v,path_in=path_temp)


100%|██████████| 208/208 [00:03<00:00, 63.33it/s] 


In [ ]:
# LIDC_nodul 变nii
# a = os.listdir(path6)
# a.sort()
# a = glob(path6+'/*.dcm')

# a.sort()

# import sys
# sys.path.append('/home/extends/cxt/model')
# from tools import fold2nii

# fold2nii(a,path6)

In [4]:
# # 取出一个volumn到一个文件夹,构造hospital_testvolumn
# for name in ['origin','rebi_mask','mask']:
#    a = os.listdir('hospital_data_clean/dcm/{0}'.format(name))
#    a.sort()
#    list_res = []
#    for i in tqdm(range(1,2)):
#       list_temp = [j for j in a if 'id0{}'.format(i) in j]
#       list_temp.sort(key=lambda x:int(x.split('_')[1].split('.')[0]))
#       list_res+=list_temp
#       break
#    a = list_res
#    c = 'hospital_data_clean/dcm/{0}'.format(name) + '/' + pd.Series(a) 
#    c.tolist()


#    for path_ in c:
#       try:
#          os.makedirs('hospital_testvolumn/{0}/'.format(name))
#       except:
#          copyfile(path_,'hospital_testvolumn/{0}/'.format(name)+path_.rsplit('/',1)[1])

  0%|          | 0/4 [00:00<?, ?it/s]


In [22]:
# 取出1/10作为训练集，另外9/10作为测试集,做成json的格式记录,构造finetune数据集
name = 'origin'

name2 = 'mask_bone'

allpic = glob('hospital_data_clean/png/{0}/*.png'.format(name))

sample_num = int(len(allpic)/10)
train_pic = random.sample(allpic, sample_num)
test_pic = list(set(allpic) - set(train_pic))
if name == 'origin':
    with open('hospital_data_clean/png/finetune/train.json','w') as f:
        json.dump(train_pic, f, indent=4)
    
    with open('hospital_data_clean/png/finetune/test.json','w') as f:
        json.dump(test_pic, f, indent=4)

# copy
import shutil

for i in tqdm(train_pic):
    shutil.copy(i,'hospital_data_clean/png/finetune/train/{0}/'.format(name))
    shutil.copy(i.replace(name, name2),'hospital_data_clean/png/finetune/train/mask/')

for j in tqdm(test_pic):
    shutil.copy(j,'hospital_data_clean/png/finetune/test/{0}/'.format(name))
    shutil.copy(j.replace(name, name2),'hospital_data_clean/png/finetune/test/mask/')

100%|██████████| 6307/6307 [00:37<00:00, 169.09it/s]


In [10]:
# CTDircadb1 数据集变成nii文件
path1 = 'CTDircadb1/test/pred'
path2 = 'CTDircadb1/test/mask'
path3 = 'CTDircadb1/test/origin'
for path_temp in [path1,path2,path3]:
    
    path_ = glob(path_temp+'/*.png')
    path_.sort()
    count = 0
    for pic in tqdm(path_):
        if count == 0:
            
            a = cv2.imread(pic,1) 
            a = cv2.cvtColor(a,cv2.COLOR_BGR2GRAY)
            a = a[np.newaxis,:]
            
        else:
            b = cv2.imread(pic,1)
            b = cv2.cvtColor(b,cv2.COLOR_BGR2GRAY)
            b = b[np.newaxis,:]
            a = np.vstack((a,b))
            
        count += 1

    out = sitk.GetImageFromArray(a)
    sitk.WriteImage(out, path_temp+'.nii.gz')
        

100%|██████████| 282/282 [00:06<00:00, 42.72it/s] 


In [1]:
## rebi_mask/origin/mask 转nii
import sys,os
import pandas as pd

sys.path.append('/home/extends/cxt/model')
from tools import fold2nii
for name in ['rebi_mask','origin','mask','mask_bone']:
    path_ = 'hospital_testvolumn/{0}'.format(name)
    q = os.listdir(path_)
    q.sort(key=lambda x:int(x.split('_')[1].split('.')[0]))
    d = path_ + '/' + pd.Series(q) 
    d = d.to_list()
    fold2nii(d,path_)

100%|██████████| 305/305 [00:06<00:00, 47.40it/s] 
